<a href="https://colab.research.google.com/github/PoiZeX/CloudCourse/blob/main/ex6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from ipywidgets.widgets.widget_button import ButtonStyle
#@title
import time
from ipywidgets.widgets.widget_style import Style
from zmq.constants import NULL
from firebase import firebase
import IPython
from google.colab import output
import random
import ipywidgets as widgets
from ipywidgets import VBox, HBox, HTML, Layout, Box, Label
import matplotlib.pyplot as plt
import math
import re




##########################################################################################################################
############################################### CONFIGURATION <-  ########################################################
##########################################################################################################################

FIREBASE_PATH = 'https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/'
style = {'description_width': '150px'} 
box_layout = widgets.Layout(display='flex',  flex_flow='column',  align_items='center',  width='100%')
################################################################################
#################################### CLASSES ###################################
################################################################################
class Question():
  answers = []
  def __init__(self, question, lst_options, correct):
    self.question = question
    self.answers = lst_options[1:]
    self.correct = correct
  
  def isCorrect(self, str_option):
    correctAns = self.answers[int(self.correct)-1]
    return str_option.strip() == correctAns.strip()
  
  def get_random_answers(self):
      return random.sample(self.answers, len(self.answers))
  
  def get_question_str(self):
    return self.question

################################################################################
class QuestionsDB():
  lst_questions = []
  current_question = None
  original_len = 0

  def __init__(self, lst_questions):
    self.lst_questions = [Question(item['question'], item['answers'], item['correctAnswer']) for item in lst_questions if item is not None]
    self.original_len = QuestionsDB.get_num_of_questions(self)

  def get_random_question(self):
    global randomQuestionAndAnswer, answers, question, radioButtonLst
    randomQuestionAndAnswer = NULL # we want to generate new item...
    if self.lst_questions:
      random_question = random.sample(self.lst_questions, 1)[0]
      self.lst_questions.remove(random_question)
      self.current_question = random_question
      return random_question
    else:
      self.current_question = []
      return
   
  def add_question_to_DB(self):
    pass

  def remove_question_from_DB(self):
    pass

  def get_num_of_questions(self):
    return len(self.lst_questions)
  
  def get_original_len(self):
    return self.original_len

  def get_current_question_obj(self):
    return self.current_question
################################################################################
class User():
  def __init__(self, username, password, role):
    self.username = username
    self.password = password
    self.role = role
  
  def isAdmin(self):
    return self.role == "Admin"
  
  def getName(self):
    return self.username

##########################################################################################################################
#############################################  FIREBASE #################################################################
##########################################################################################################################
def get_questions_from_db():
  global questionsFromDB
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  res = FBconn.get('/Questions/',None)

  if res:
    # insert questions to DB class
    questionsFromDB = QuestionsDB([question for question in res])


##########################################################################################################################
######## GLOBAL VARIABLES
##########################################################################################################################
questionsFromDB = None
btn_go_back = NULL
current_user = NULL
points = 0
quantity_of_questions=0
quantity_of_correct_questions=0
list_of_stats = []  # will be filled with number of attempts for EACH question
currentSelectedAns = ""
finalSelectedBtn = widgets.Button(description="") # null object
question = ""
buttonsLst = []
#For Admin Form 
#--->
cb_correctAnswerId = NULL
tb_question= NULL
tb_answer1 = NULL
tb_answer2 = NULL
tb_answer3 = NULL
tb_answer4 = NULL
offset = 0
#<---

##########################################################################################################################
########################################################## UTILS #########################################################
##########################################################################################################################
from enum import Enum
class WidgetEnum(Enum):
    BUTTON = 1
    LABEL = 2     
    TEXTBOX = 3 

def show_h2_center_text(text):
  """
  Generic function for h2 centered text
  """
  display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
    f"<h2>{text}</h2><br/>"))

def show_h4_center_text(text):
  """
  Generic function for h4 centered text
  """
  display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
    f"<h4>{text}</h4><br/>"))
  

def create_widget(widget_enum, description, button_style = "", layout="", onclick=""):
  b_layout = widgets.Layout( width='auto', height='auto', min_width="150px")
  b_style={'white_space': 'pre-wrap'}
  style = {'description_width': '150px'} 
  
  obj = None
  if widget_enum == WidgetEnum.BUTTON:
    if layout != "":
      b_layout = layout
    obj = widgets.Button(description = str(description), button_style = button_style,layout = b_layout, style=b_style)
    if onclick != "":
        obj.on_click(onclick)

  elif widget_enum == WidgetEnum.LABEL:
    return None

  elif widget_enum == WidgetEnum.TEXTBOX:
      obj = widgets.Text(description=description, style=style) 
      
  else:
    return None

  return obj

def center_widgets(widgets_lst):
  """
  gets a list of widgets and centered them
  """
  return widgets.HBox(children=widgets_lst,layout=box_layout)

def show_go_back(back_event):
  if btn_go_back != NULL:
    display(btn_go_back)
  btn_go_back.on_click(back_event)


##########################################################################################################################
################################### RESULTS SECTION -> SHOWS THE RESULTS AND GRAPH <-  ###################################
##########################################################################################################################


def show_statistics_chart():
  """
  Generate and show statistics plot. 
  The statistics are number of attempts for each question
  """
  global list_of_stats
  list_of_stats.insert(0, 0) # normalize the data
  fig, ax = plt.subplots() # configure
  
  ax.plot(list_of_stats) # make the line plot

  # mark the dots
  for i, stat in enumerate(list_of_stats):
    if stat != 0:
        ax.scatter(i, stat, color='green', marker='o')

  # make data readable (start from 1)
  ax.set_xticks(range(1, len(list_of_stats))) 
  ax.set_yticks(range(1, 5))

  ax.set_title("Your game statistics")
  ax.set_ylabel("Number of attempts")
  ax.set_xlabel("Question #")
  
  plt.show()

def show_results_page():
    good_words = ""
    badge_src = ""
    total_score_to_earn = 3*questionsFromDB.get_num_of_questions()
    name = current_user.getName()
    if total_score_to_earn < 2:
      good_words = f"{name}, You are AWESOME!"
      badge_src = "https://thumbs.dreamstime.com/z/gold-medal-st-place-badge-symbol-victory-sport-business-gold-medal-st-place-badge-symbol-victory-sport-business-128259723.jpg"
    elif total_score_to_earn - points < 15:
      good_words = f"{name}, You are great! Try again to beat yourself "
      badge_src = "https://c8.alamy.com/comp/JE56PM/silver-2st-place-medal-vector-metal-realistic-badge-with-second-placement-JE56PM.jpg"
    else:
      good_words = f"{name}, You are good, but need more practice. try out our courses and play again"
      badge_src = "https://thumbs.dreamstime.com/b/award-badge-third-place-red-ribbon-isolated-medal-pin-symbol-realistic-d-design-victory-trophy-sign-number-white-147089746.jpg"
    succ_percentages = round((points/total_score_to_earn)*100)
    display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
    f"<h2 style='color: red'><img src='https://www.animatedimages.org/data/media/492/animated-fireworks-image-0087.gif' style='width: 50px; height: 50px'/>"+
                                f"Game over!<img src='https://www.animatedimages.org/data/media/492/animated-fireworks-image-0087.gif' style='width: 50px; height: 50px'/></h2>"+
    f"<h4><u>Your score</u></h4><br/><strong>Points:</strong> {points} | "+
    f"<strong>Correct answers:</strong> {quantity_of_correct_questions}"
    f"<br/><strong>You could earn up to</strong> {total_score_to_earn} points."+
    f"<br/><strong>Your success rate is:</strong> {succ_percentages}%"+
    
    f"<h4 style='color: #00bfff;'>{good_words}</h4><br/>"+
    f"<div>You earn a new badge: <img src={badge_src} style='width: 150px; height: 150px; display:block; margin:auto; '/></div></div>"))
    show_statistics_chart()

##################################################################################################################################
################################### GAME SECTION -> QUESTION PROGRESS FROM START TO FINISH <-  ###################################
##################################################################################################################################

def game_over():
  output.clear()
  show_results_page()

  
def save_answer(b):
  """
  On click on some answer
  """
  global currentSelectedAns, finalSelectedBtn
  currentSelectedAns = str(b.description)  # save the answer to compare
  finalSelectedBtn = b  # save the button object to mark it later
  on_answer_button_clicked(b)
  update_total_questions_quantity()
  if questionsFromDB.get_num_of_questions() == 11 : # LIDOR
      game_over()
      return

def display_questions():
  """
  Create questions page elemnts and show them
  """
  global randomQuestionAndAnswer, answers, question, radioButtonLst

  current_question = questionsFromDB.get_current_question_obj()

  display(IPython.display.HTML(f"<h2 style='display: flex; justify-content: center; align-items: center;'>{current_question.get_question_str()}</h2>"))
  points_label = HTML(f"<strong>Points:</strong> {points}")
  correct_label = HTML(f"<strong>Correct Questions:</strong> {quantity_of_correct_questions}")
  total_label = HTML(f"<strong>Out Of:</strong> {quantity_of_questions}")

  box = widgets.HBox(children=[points_label, correct_label, total_label],layout=box_layout)
  display(box)
  
  buttonsLst = [widgets.Button(description = str(answer), layout = widgets.Layout( width='auto', height='auto'),
                    style={'white_space': 'pre-wrap'}
                  ) for answer in current_question.get_random_answers()]

  for btn in buttonsLst:
    btn.on_click(save_answer)
    btn.style.button_color = 'orange'

  left_box = VBox([buttonsLst[0], buttonsLst[1]])
  right_box = VBox([buttonsLst[2], buttonsLst[3]])
  
  box = widgets.HBox(children=[HBox([left_box, right_box])],layout=box_layout)
  display(box)

def show_questions_page():
  """
  Generate a new question
  """
  global show_go_back
  output.clear()
  #get_new_random_question_and_answers()
  questionsFromDB.get_random_question()
  display_questions()
  btn_go_back.description = "Exit game"
  show_go_back(show_menu_page)  

def on_answer_button_clicked(b):
    """
    The true logic for answer button clicked
    """
    global finalSelectedBtn, list_of_stats

    if currentSelectedAns is NULL:
      print("Please select answer")
      return
    list_of_stats[questionsFromDB.get_original_len() - questionsFromDB.get_num_of_questions() - 1] += 1

    # check if user's answer is correct

    
    if questionsFromDB.current_question.isCorrect(currentSelectedAns):
        update_points(3)
        update_total_correct_questions_quantity()
        print("Correct!")
        finalSelectedBtn.style.button_color = 'lightgreen'

        # start timer and refresh
        time.sleep(1.5)
        show_questions_page()
    else:
        update_points(-1)
        finalSelectedBtn.style.button_color = 'red'
        display(IPython.display.HTML('<br/><h4>Incorrect answer<br/>Try Again! You can do it!<h4/><br/>'))

def update_points(new_points):
    """
    Update and display current points
    """
    global points
    points += new_points
    display(IPython.display.Markdown(f"**New Points:** {points}"))
    correct_label = IPython.display.Markdown(f"**Correct Questions:** {quantity_of_correct_questions}")
    total_label = IPython.display.Markdown(f"**Out Of:** {quantity_of_questions}")
    display(correct_label)
    display(total_label)

def update_total_questions_quantity():
    global quantity_of_questions
    quantity_of_questions+=1

def update_total_correct_questions_quantity():
    global quantity_of_correct_questions
    quantity_of_correct_questions+=1



##########################################################################################################################
##########################################################ADMIN ##########################################################
##########################################################################################################################
def insert_new_question_to_db(b):
  """
  Inserting new question. Return true if succeeded
  """
  global offset  
  # Connect to DB
  try:
    # TODO: check validation (not empty, no repeats)
    data_to_upload = {
      'question' : tb_question.value,
      'answers' : [None, tb_answer1.value, tb_answer2.value, tb_answer3.value, tb_answer4.value],
      'correctAnswer' : cb_correctAnswerId.value
    }
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    result = FBconn.put("/Questions", name=f"{len(questionsFromDB)+offset+1}", data=data_to_upload)
    offset += 1
    print("Success")
    return True
  except Exception as e:
    print(f"Can't insert into DB\n{e}")
    return False


def show_form():
    """
    Present a form for adding new questions into DB.
    """
    global cb_correctAnswerId, tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, offset
    output.clear()
    cb_correctAnswerId = widgets.Dropdown(options=['1','2','3','4'],  description='Correct answer Number:',
        disabled=False, style=style)

    tb_question = widgets.Text(description='Question:', style=style,)
    tb_answer1 = widgets.Text(description='Answer #1:', style=style,)
    tb_answer2 = widgets.Text(description='Answer #2:', style=style,)
    tb_answer3 = widgets.Text(description='Answer #3:', style=style,)
    tb_answer4 = widgets.Text(description='Answer #4:', style=style,)
    
    btn_insert = widgets.Button(description='Insert', disabled=False, 
                            button_style='success', 
                            layout=Layout(margin="20px 0px 0px 140px"))
    btn_insert.on_click(insert_new_question_to_db)

    display(VBox([tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, cb_correctAnswerId, btn_insert]))
    show_go_back(show_menu_page)


##########################################################################################################################
########################################################## MENU-PAGE #####################################################
##########################################################################################################################
def show_menu_page(b=NULL):
    """
    Present the main menu
    """
    output.clear()
    global btn_go_back, list_of_stats
    btn_questions = widgets.Button(description="Questions")
    btn_classes = widgets.Button(description="Classes")
    btn_leaderboard = widgets.Button(description="Leaderboard")
    btn_help = widgets.Button(description="Help")
    btn_admin = widgets.Button(description="Add Questions (Admin only)", 
                               layout=Layout(width="200px"))
    btn_go_back = widgets.Button(description="Back", button_style="primary",layout=Layout(width="80px"))
    
    def on_questions_clicked(b):
        global list_of_stats
        output.clear()
        list_of_stats = [0]*(questionsFromDB.get_num_of_questions())
        show_questions_page()
      

    def on_leaderboard_clicked(b):
        output.clear()
        # 
        display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
        f"<h2 style='color: red'><img src='https://media.tenor.com/8QEKfyK2RUcAAAAM/rotating-star.gif' style='width: 50px; height: 50px'/>"+
                                    f"Leaderboard!<img src='https://media.tenor.com/8QEKfyK2RUcAAAAM/rotating-star.gif' style='width: 50px; height: 50px'/></h2>"+
        f"</h2></div>"))
        display(IPython.display.HTML(f"<h4>Coming up soon</h4>"))
        show_go_back(show_menu_page)
        
    def on_classes_clicked(b):
        output.clear()
        display(IPython.display.HTML(f"<h2>Classes page</h2>"))
        display(IPython.display.HTML(f"<h4>To gain access to our exceptional"+
        " classes, kindly get in touch with our customer service team to arrange payment.</h4>"))
        show_go_back(show_menu_page)

    
    def on_help_clicked(b):
        output.clear()
        display(IPython.display.HTML(f"<h2>Help page</h2>"))
        display(IPython.display.HTML(f"<h4>Answer as many questions as possible "+ 
          f"with the fewest attempts.<br/>Each correct answer is worth 3 points, "+
          f"but each incorrect answer will cost you 1 point</h4>"))
        display(IPython.display.HTML(f"<br/><br/><h4>Have Fun!</h4>"))
        show_go_back(show_menu_page)


    # Assign callbacks to buttons
    btn_questions.on_click(on_questions_clicked)
    btn_classes.on_click(on_classes_clicked)
    btn_help.on_click(on_help_clicked)
    btn_admin.on_click(show_form)
    btn_leaderboard.on_click(on_leaderboard_clicked)
    # Display Menu buttons
    if current_user.isAdmin():
      box = widgets.HBox(children=[btn_questions,btn_leaderboard, btn_classes, btn_help, btn_admin], layout=box_layout)
    else:
      box = widgets.HBox(children=[btn_questions,btn_leaderboard, btn_classes, btn_help], layout=box_layout)
    display(box)




##########################################################################################################################
########################################################## LOGIN #########################################################
##########################################################################################################################


def show_login_page():
    """
    Create and show the login page elements
    """
    def handle_login_clicked(b):
          validate_login(tb_username.value, tb_password.value)

    show_h2_center_text("Welcome!")
    tb_username = create_widget(WidgetEnum.TEXTBOX,'Username')
    tb_password = create_widget(WidgetEnum.TEXTBOX,'Password')
    btn_login = create_widget(WidgetEnum.BUTTON, 'Login', button_style='success', onclick=handle_login_clicked)
    
   
    display(VBox([center_widgets([tb_username, tb_password, btn_login])]))


    def sign_up(username, password):
      """
      Insert new user to DB
      """
      # Connect to DB
      try:
        # TODO: check validation (not empty, no repeats)
        data_to_upload = {
          'Username' : f'{username}',
          'Password' : f'{password}',
          'Role' : 'User'
        }
        FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
        result = FBconn.post("/Users", data=data_to_upload)

        display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
        f"<h4>You registered successfully!<br/>Next time you can login with your details</h4>"))
        time.sleep(3)
        show_menu_page() # Auto navigate to 

      except Exception as e:
        print(f"Can't insert into DB\n{e}")
        return False

    def is_user_Exist(username):
      FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
      users = FBconn.get('/Users/', None)
      for user_id, user_data in users.items():
          if user_data['Username'] == username:
              return True 
      return False

      
    def validate_login(username, password):
      """
      Validate the login info against DB, and suggest to register if need, or starting the game
      """
      def get_user():
        global current_user
        FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
        users = FBconn.get('/Users/', None)
        
        for user_id, user_data in users.items():
            if user_data['Username'] == username and user_data['Password'] == password:
                current_user = User(username, password, user_data['Role'])
                return True 
        return False


      if not username or not password:
          show_h4_center_text("Please insert username & password")
      else:
          if is_user_Exist(username):#added "not" to match the correct case[SHARON] 
                                    #[LIDOR] - This should be without the "not". I want to make sure the user is exist but the password is not
            res = get_user() 
            if res:  # user and password corrects
              show_menu_page() 
            else:
              show_h4_center_text("Incorrect password")
              return
      
        

          else:
            # show message and suggest to register
            show_h4_center_text("The username / password are incorrect")
            show_h4_center_text("If you are new, please register")
            def handle_signup_clicked(b):
              sign_up(username, password)

            btn_register = create_widget(WidgetEnum.BUTTON, 'Register', button_style='primary', onclick=handle_signup_clicked)
            display(VBox([center_widgets([btn_register])]))



        


##########################################################################################################################
########################################################## MAIN ##########################################################
##########################################################################################################################
get_questions_from_db()
show_login_page()
